In [1]:
require 'xlua'
require 'optim'
require 'nn'
require 'cunn'
require 'csvigo'
dofile './provider.lua'
c = require 'trepl.colorize'


cmd_params = {}
----- from the opt settings ------
cmd_params.save = 'logs_stl10test/vgg_baseline'
cmd_params.batchSize = 128
cmd_params.learningRate = 1
cmd_params.learningRateDecay = 1e-7
cmd_params.weightDecay = 0.0005
cmd_params.momentum = 0.9
cmd_params.epoch_step = 25
cmd_params.model = '../RegionMatching_cifar10/models/vgg_bn_drop'  --for cifar10
--cmd_params.model = '../RegionMatching_cifar100/logs_cifar100/1_vgg_bn_drop'  --for cifar100
cmd_params.max_epoch = 300
cmd_params.backend = 'nn'
cmd_params.type = 'cuda'
----------------------------------
cmd_params.name = 'experiment'
cmd_params.gpumode = 1
cmd_params.gpu_setDevice = 1

In [2]:
function cast(t)
   if cmd_params.type == 'cuda' then
      require 'cunn'
        gpumode = cmd_params.gpumode
        if gpumode==1 then
            cutorch.setDevice(cmd_params.gpu_setDevice)
        end
      return t:cuda()
   elseif cmd_params.type == 'float' then
      return t:float()
   elseif cmd_params.type == 'cl' then
      require 'clnn'
      return t:cl()
   else
      error('Unknown type '..cmd_params.type)
   end
end

In [3]:
local seed = 1234567890
torch.manualSeed(seed)

In [4]:
----Data Augmentation
function data_aug(input)
      local bs = input:size(1)
      local flip_mask = torch.randperm(bs):le(bs/2)
      for i=1,input:size(1) do
        if flip_mask[i] == 1 then image.hflip(input[i], input[i]) end
      end
    return input
end

In [5]:
----Initiation

--1. Data loading
print(c.blue '==>' ..' loading data')
provider = torch.load '/media/sjvision/DATASETDISK/atest/stl10_matlab_testeddirectlyforaccuracy/t7_files/stl_trainnorm.t7'  --for cifar10
--provider = torch.load'/media/sjvision/DATASETDISK/cifars/cifar100/cifar100_whitened.t7'
provider.trainData.data = provider.trainData.data:float()
provider.testData.data = provider.testData.data:float()
----- Load the appropriate data ------

--2. Model creation
model_wts = torch.load('../RegionMatching_cifar10/logs/vgg_withHFLIP/model.net') --for cifar10
--model_wts = torch.load('../RegionMatching_cifar100/logs_cifar100/1_vgg_bn_drop/model_final.net')  --for cifar100
model = nn.Sequential()
model:add(cast(nn.Copy('torch.FloatTensor', torch.type(cast(torch.Tensor())))))
model:add(model_wts)
--[[
if cmd_params.backend == 'cudnn' then
   require 'cudnn'
   cudnn.convert(model:get(2), cudnn)
end
]]--
parameters,gradParameters = model:getParameters()

--3. Criterion
print(c.blue'==>' ..' setting criterion')
criterion = cast(nn.CrossEntropyCriterion())

--4. Testing and saving
print('Will save at '..cmd_params.save)
paths.mkdir(cmd_params.save)
testLogger = optim.Logger(paths.concat(cmd_params.save, 'test.log'))
testLogger:setNames{'% mean class accuracy (train set)', '% mean class accuracy (test set)'}
testLogger.showPlot = false

--5. Learning settings
print(c.blue'==>' ..' configuring optimizer')
optimState = {
  learningRate = cmd_params.learningRate,
  weightDecay = cmd_params.weightDecay,
  momentum = cmd_params.momentum,
  learningRateDecay = cmd_params.learningRateDecay,
}


==> loading data	


==> setting criterion	


Will save at logs_stl10test/vgg_baseline	


==> configuring optimizer	


In [6]:
parameters:size()

In [7]:
function test()
  -- disable flips, dropouts and batch normalization
  model:evaluate()
  print(c.blue '==>'.." testing")
  local confusion = optim.ConfusionMatrix(10)
    
  local bs = 1
  local nsamples = provider.testData.data:size(1)
  print(nsamples)
  local fv_output = torch.zeros(nsamples,512)
  
  for i=1,nsamples,bs do 
    local outputs = model:forward(provider.testData.data:narrow(1,i,bs))
-------1. Analysing the right and wrong test samples   
    confusion:batchAdd(outputs, provider.testData.labels:narrow(1,i,bs))
    confusion:updateValids()
    --[[
    if confusion.totalValid==0 then
        itorch.image(testData:narrow(1,i,bs))
        print(i)
    end
    print('Test accuracy:', confusion.totalValid * 100)
    confusion:zero()
    --]]
------2. Extracting the feature vector
    --fv_output[i]=model:get(2):get(54):get(4).output:float()
  end
  --fv_output = fv_output:totable()
  --csvigo.save('logs_stanfordaction40/vgg_baseline_correctfv.txt',fv_output)
  print('Test accuracy:', confusion.totalValid * 100)
end


 14991946
[torch.LongStorage of size 1]



In [8]:
for i=1,1 do
  test()
end

==> testing	
5000	


Test accuracy:	45.34	
